In [1]:
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime, timedelta

In [2]:
print("Current working directory:", os.getcwd())
os.chdir(r'..')
print("Current working directory:", os.getcwd())

Current working directory: c:\Users\Lukasz Pindus\VS Code Python\car_price_analysis\notebooks
Current working directory: c:\Users\Lukasz Pindus\VS Code Python\car_price_analysis


In [3]:
#df = pd.read_csv('data/merged01.csv')
df = pd.read_csv('C:/Users/Lukasz Pindus/VS Code Python/car_price_analysis/data/merged01.csv', dtype={
    'Pojemność baterii': str,
    'Autonomia': str,
    'Średnie zużycie': str,
    'Kondycja baterii': str,
    'Typ złącza ładowania': str,
    'advert_date': str
})

In [ ]:
print(df.columns)

In [ ]:
df.info()

In [ ]:
df[df['advert_id'].notnull()]['advert_id'].head(10)

In [ ]:
df[df['Liczba silników'].notnull()]['Liczba silników'].unique()

In [4]:
# # Extract number and unit
# df[['battery_capacity_value', 'battery_capacity_unit']] = df['Pojemność baterii'].str.extract(r'(\d+\.?\d*)\s*(\w+)')

# # Convert value to float for analysis
# df['battery_capacity_value'] = df['battery_capacity_value'].astype(float)

# df = df.drop('Pojemność baterii', axis=1)

def clean_battery_capacity(series, expected_unit='kWh'):
    # Extract number and unit
    extracted = series.str.extract(r'(?P<value>\d+\.?\d*)\s*(?P<unit>\w+)', expand=True)

    # Check for unexpected units
    non_null_units = extracted['unit'].dropna()
    unexpected_units = non_null_units[~non_null_units.eq(expected_unit)].unique()
    if len(unexpected_units) > 0:
        raise ValueError(f"Unexpected units found: {unexpected_units.tolist()}")

    # Return numeric series
    return extracted['value'].astype(float)

df['Pojemność_baterii_kWh'] = clean_battery_capacity(df['Pojemność baterii'])

df = df.drop('Pojemność baterii', axis=1)

In [5]:
def clean_range_column(series, expected_unit='km'):
    # Extract numeric value and unit
    extracted = series.str.extract(r'(?P<value>\d+\.?\d*)\s*(?P<unit>\w+)', expand=True)

    # Validate unit
    non_null_units = extracted['unit'].dropna()
    unexpected_units = non_null_units[~non_null_units.eq(expected_unit)].unique()
    if len(unexpected_units) > 0:
        raise ValueError(f"Unexpected units in Autonomia: {unexpected_units.tolist()}")

    return extracted['value'].astype(float)

df['Autonomia_km'] = clean_range_column(df['Autonomia'])

df = df.drop('Autonomia', axis=1)

In [6]:
def clean_engine_displacement(series, expected_unit='cm3'):
    # Remove thousand separators (spaces)
    cleaned = series.str.replace(' ', '', regex=False)

    # Extract numeric value and unit
    extracted = cleaned.str.extract(r'^(?P<value>\d+\.?\d*)\s*(?P<unit>[a-zA-Z/]+)$', expand=True)

    # Validate unit
    non_null_units = extracted['unit'].dropna()
    unexpected_units = non_null_units[~non_null_units.eq(expected_unit)].unique()
    if len(unexpected_units) > 0:
        raise ValueError(f"Unexpected units in engine displacement: {unexpected_units.tolist()}")

    return extracted['value'].astype(float)

# Apply it
df['engine_displacement_cm3'] = clean_engine_displacement(df['Pojemność skokowa'])
df = df.drop('Pojemność skokowa', axis=1)

In [7]:
def clean_moc_column(series):
    # Remove any spaces (in case of '1 000 kW')
    cleaned = series.str.replace(' ', '', regex=False)

    # Extract numeric value and unit
    extracted = cleaned.str.extract(r'(?P<value>\d+\.?\d*)\s*(?P<unit>\w+)', expand=True)

    # Normalize units to lowercase for comparison
    extracted['unit'] = extracted['unit'].str.lower()

    # Allowed units
    expected_unit = ['km', 'kw']

    # Check for unexpected units
    non_null_units = extracted['unit'].dropna()
    unexpected_units = non_null_units[~non_null_units.isin(expected_unit)].unique()
    if len(unexpected_units) > 0:
        raise ValueError(f"Unexpected units in 'Moc': {unexpected_units.tolist()}")

    # Convert to float
    extracted['value'] = extracted['value'].astype(float)

    # Convert kW to KM (1 kW = 1.35962 KM), KM stays as-is
    extracted['value'] = extracted.apply(
        lambda row: row['value'] * 1.35962 if row['unit'] == 'kw' else row['value'],
        axis=1
    )

    return extracted['value']

df['power_hp'] = clean_moc_column(df['Moc'])

df = df.drop('Moc', axis=1)

In [8]:
def clean_co2_emissions_column(series, expected_unit='g/km'):
    # Remove any extra spaces (e.g., '96 g/km')
    cleaned = series.str.replace(' ', '', regex=False)

    # Extract numeric value and unit
    extracted = cleaned.str.extract(r'(?P<value>\d+\.?\d*)\s*(?P<unit>.+)', expand=True)

    # Validate unit
    non_null_units = extracted['unit'].dropna()
    unexpected_units = non_null_units[~non_null_units.eq(expected_unit)].unique()
    if len(unexpected_units) > 0:
        raise ValueError(f"Unexpected units in 'Emisja CO2': {unexpected_units.tolist()}")

    # Return numeric values as float
    return extracted['value'].astype(float)

df['co2_emissions_gpkm'] = clean_co2_emissions_column(df['Emisja CO2'])

df = df.drop('Emisja CO2', axis=1)

In [9]:
def clean_urban_fuel_column(series, expected_unit='l/100km'):
    # Remove spaces just in case (e.g., '7.8 l/100km' → '7.8l/100km')
    cleaned = series.str.replace(' ', '', regex=False)

    # Extract numeric value and unit
    extracted = cleaned.str.extract(r'(?P<value>\d+\.?\d*)\s*(?P<unit>.+)', expand=True)

    # Validate unit
    non_null_units = extracted['unit'].dropna()
    unexpected_units = non_null_units[~non_null_units.eq(expected_unit)].unique()
    if len(unexpected_units) > 0:
        raise ValueError(f"Unexpected units in 'Spalanie W Mieście': {unexpected_units.tolist()}")

    # Return numeric value as float
    return extracted['value'].astype(float)

df['urban_fuel_consumption_l_per_100km'] = clean_urban_fuel_column(df['Spalanie W Mieście'])

df = df.drop('Spalanie W Mieście', axis=1)

In [10]:
def clean_extraurban_fuel_column(series, expected_unit='l/100km'):
    # Remove spaces just in case
    cleaned = series.str.replace(' ', '', regex=False)

    # Extract numeric value and unit
    extracted = cleaned.str.extract(r'(?P<value>\d+\.?\d*)\s*(?P<unit>.+)', expand=True)

    # Validate unit
    non_null_units = extracted['unit'].dropna()
    unexpected_units = non_null_units[~non_null_units.eq(expected_unit)].unique()
    if len(unexpected_units) > 0:
        raise ValueError(f"Unexpected units in 'Spalanie Poza Miastem': {unexpected_units.tolist()}")

    # Return numeric value as float
    return extracted['value'].astype(float)

df['extraurban_fuel_consumption_l_per_100km'] = clean_extraurban_fuel_column(df['Spalanie Poza Miastem'])

df = df.drop('Spalanie Poza Miastem', axis=1)

In [11]:
def clean_mileage_column(series, expected_unit='km'):
    # Remove thousand separators (spaces)
    cleaned = series.str.replace(' ', '', regex=False)

    # Extract numeric value and unit
    extracted = cleaned.str.extract(r'(?P<value>\d+)\s*(?P<unit>\w+)', expand=True)

    # Validate unit
    non_null_units = extracted['unit'].dropna()
    unexpected_units = non_null_units[~non_null_units.eq(expected_unit)].unique()
    if len(unexpected_units) > 0:
        raise ValueError(f"Unexpected units in 'Przebieg': {unexpected_units.tolist()}")

    # Return as integer (mileage usually doesn't need decimals)
    return extracted['value'].astype(int)

df['mileage_km'] = clean_mileage_column(df['Przebieg'])

df = df.drop('Przebieg', axis=1)

In [12]:
def clean_avg_energy_consumption_column(series, expected_unit='kWh/100km'):
    # Remove any spaces (e.g. '15 kWh/100km' → '15kWh/100km')
    cleaned = series.str.replace(' ', '', regex=False)

    # Extract numeric value and unit
    extracted = cleaned.str.extract(r'(?P<value>\d+\.?\d*)\s*(?P<unit>.+)', expand=True)

    # Validate unit
    non_null_units = extracted['unit'].dropna()
    unexpected_units = non_null_units[~non_null_units.eq(expected_unit)].unique()
    if len(unexpected_units) > 0:
        raise ValueError(f"Unexpected units in 'Średnie zużycie': {unexpected_units.tolist()}")

    # Return numeric value as float
    return extracted['value'].astype(float)

df['average_energy_consumption_kwh_per_100km'] = clean_avg_energy_consumption_column(df['Średnie zużycie'])

df = df.drop('Średnie zużycie', axis=1)

In [13]:
def clean_battery_health_column(series, expected_unit='%'):
    # Remove spaces (e.g. '98 %' → '98%')
    cleaned = series.str.replace(' ', '', regex=False)

    # Extract numeric value and unit
    extracted = cleaned.str.extract(r'(?P<value>\d+\.?\d*)\s*(?P<unit>[%])', expand=True)

    # Validate unit
    non_null_units = extracted['unit'].dropna()
    unexpected_units = non_null_units[~non_null_units.eq(expected_unit)].unique()
    if len(unexpected_units) > 0:
        raise ValueError(f"Unexpected units in 'Kondycja baterii': {unexpected_units.tolist()}")

    # Return numeric value as float
    return extracted['value'].astype(float)

df['battery_health_percent'] = clean_battery_health_column(df['Kondycja baterii'])

df = df.drop('Kondycja baterii', axis=1)

In [14]:
def clean_max_electric_power_column(series, expected_unit='HP'):
    # Remove spaces just in case (e.g., '120 HP' → '120HP')
    cleaned = series.str.replace(' ', '', regex=False)

    # Extract numeric value and unit
    extracted = cleaned.str.extract(r'(?P<value>\d+\.?\d*)\s*(?P<unit>\w+)', expand=True)

    # Validate unit
    non_null_units = extracted['unit'].dropna()
    unexpected_units = non_null_units[~non_null_units.eq(expected_unit)].unique()
    if len(unexpected_units) > 0:
        raise ValueError(f"Unexpected units in 'Elektryczna moc maksymalna HP': {unexpected_units.tolist()}")

    # Return numeric value as float
    return extracted['value'].astype(float)

df['max_electric_power_hp'] = clean_max_electric_power_column(df['Elektryczna moc maksymalna HP'])

df = df.drop('Elektryczna moc maksymalna HP', axis=1)

In [15]:
df['price'] = (
    df['price']
    .str.replace(' ', '', regex=False)
    .str.replace(',', '.', regex=False)
    .apply(pd.to_numeric, errors='coerce')  # will set invalid to NaN
)

In [16]:
# Polish to numeric month mapping
month_map = {
    'stycznia': '01',
    'lutego': '02',
    'marca': '03',
    'kwietnia': '04',
    'maja': '05',
    'czerwca': '06',
    'lipca': '07',
    'sierpnia': '08',
    'września': '09',
    'października': '10',
    'listopada': '11',
    'grudnia': '12'
}

# Replace Polish month names with numbers
for pl, num in month_map.items():
    df['advert_date'] = df['advert_date'].str.replace(pl, num, regex=False)

# Now convert to datetime
df['advert_date'] = pd.to_datetime(df['advert_date'], format='%d %m %Y %H:%M')

In [17]:
df['advert_id'] = df['advert_id'].fillna(0).astype(int)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270054 entries, 0 to 270053
Data columns (total 44 columns):
 #   Column                                         Non-Null Count   Dtype         
---  ------                                         --------------   -----         
 0   Marka pojazdu                                  270052 non-null  object        
 1   Model pojazdu                                  270054 non-null  object        
 2   Wersja                                         185336 non-null  object        
 3   Kolor                                          270054 non-null  object        
 4   Liczba drzwi                                   269066 non-null  float64       
 5   Liczba miejsc                                  252416 non-null  float64       
 6   Rok produkcji                                  270054 non-null  int64         
 7   Generacja                                      196020 non-null  object        
 8   Rodzaj paliwa                               

In [19]:
# Then restore NaNs as before
df[text_cols] = df[text_cols].replace('', pd.NA)
df[num_cols] = df[num_cols].replace(-1, pd.NA)
df[date_cols] = df[date_cols].replace(pd.Timestamp('1900-01-01'), pd.NaT)

# Fill, drop duplicates, restore — all on df directly
df[text_cols] = df[text_cols].fillna('')
df[num_cols] = df[num_cols].fillna(-1)
df[date_cols] = df[date_cols].fillna(pd.Timestamp('1900-01-01'))

df = df.drop_duplicates()

df[text_cols] = df[text_cols].replace('', pd.NA)
df[num_cols] = df[num_cols].replace(-1, pd.NA)
df[date_cols] = df[date_cols].replace(pd.Timestamp('1900-01-01'), pd.NaT)


NameError: name 'text_cols' is not defined

In [ ]:
# # 1. Separate columns by type
# text_cols = df.select_dtypes(include='object').columns
# num_cols = df.select_dtypes(include=['float64', 'int64']).columns
# date_cols = df.select_dtypes(include='datetime64[ns]').columns

# # 2. Make a temporary copy
# df_temp = df.copy()

# # 3. Fill NaNs with placeholder values
# df_temp[text_cols] = df_temp[text_cols].fillna('')
# df_temp[num_cols] = df_temp[num_cols].fillna(-1)
# df_temp[date_cols] = df_temp[date_cols].fillna(pd.Timestamp('1900-01-01'))  # placeholder date

# # 4. Drop duplicates
# df_deduped = df_temp.drop_duplicates()

# # 5. Restore original NaNs
# df_deduped[text_cols] = df_deduped[text_cols].replace('', pd.NA)
# df_deduped[num_cols] = df_deduped[num_cols].replace(-1, pd.NA)
# df_deduped[date_cols] = df_deduped[date_cols].replace(pd.Timestamp('1900-01-01'), pd.NaT)

In [ ]:
# # Fill, drop duplicates, restore — all on df directly
# df[text_cols] = df[text_cols].fillna('')
# df[num_cols] = df[num_cols].fillna(-1)
# df[date_cols] = df[date_cols].fillna(pd.Timestamp('1900-01-01'))

# df = df.drop_duplicates()

# df[text_cols] = df[text_cols].replace('', pd.NA)
# df[num_cols] = df[num_cols].replace(-1, pd.NA)
# df[date_cols] = df[date_cols].replace(pd.Timestamp('1900-01-01'), pd.NaT)


In [ ]:
# # 1. Separate columns by type
# text_cols = df.select_dtypes(include='object').columns
# num_cols = df.select_dtypes(include=['float64', 'int64']).columns
# date_cols = df.select_dtypes(include='datetime64[ns]').columns

# # 2. Make a temporary copy
# df_temp = df.copy()

# # 3. Fill NaNs with placeholder values
# df_temp[text_cols] = df_temp[text_cols].fillna('')
# df_temp[num_cols] = df_temp[num_cols].fillna(-1)
# df_temp[date_cols] = df_temp[date_cols].fillna(pd.Timestamp('1900-01-01'))  # placeholder date

In [ ]:
df_temp.shape

In [20]:
# 1. Separate columns by type
text_cols = df.select_dtypes(include='object').columns
num_cols = df.select_dtypes(include=['float64', 'int64']).columns
date_cols = df.select_dtypes(include='datetime64[ns]').columns

# Fill, drop duplicates, restore — all on df directly
df[text_cols] = df[text_cols].fillna('')
df[num_cols] = df[num_cols].fillna(-1)
df[date_cols] = df[date_cols].fillna(pd.Timestamp('1900-01-01'))

df = df.drop_duplicates()

df[text_cols] = df[text_cols].replace('', pd.NA)
df[num_cols] = df[num_cols].replace(-1, pd.NA)
df[date_cols] = df[date_cols].replace(pd.Timestamp('1900-01-01'), pd.NaT)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 226088 entries, 0 to 270053
Data columns (total 44 columns):
 #   Column                                         Non-Null Count   Dtype         
---  ------                                         --------------   -----         
 0   Marka pojazdu                                  226086 non-null  object        
 1   Model pojazdu                                  226088 non-null  object        
 2   Wersja                                         154783 non-null  object        
 3   Kolor                                          226088 non-null  object        
 4   Liczba drzwi                                   225263 non-null  object        
 5   Liczba miejsc                                  211394 non-null  object        
 6   Rok produkcji                                  226088 non-null  int64         
 7   Generacja                                      163504 non-null  object        
 8   Rodzaj paliwa                                  22

In [ ]:
df[df['Liczba miejsc'].notnull()]['Liczba miejsc'].head(10)

In [ ]:
df[df['Rodzaj paliwa'].notnull()]['Rodzaj paliwa'].unique()

In [ ]:
print(df.columns)

In [21]:
column_mapping = {
    'Marka pojazdu': 'make',
    'Model pojazdu': 'model',
    'Wersja': 'version',
    'Kolor': 'color',
    'Liczba drzwi': 'number_of_doors',
    'Liczba miejsc': 'number_of_seats',
    'Rok produkcji': 'production_year',
    'Generacja': 'generation',
    'Rodzaj paliwa': 'fuel_type',
    'Typ nadwozia': 'body_type',
    'Rodzaj koloru': 'color_type',
    'Skrzynia biegów': 'transmission',
    'Napęd': 'drive_type',
    'Kraj pochodzenia': 'country_of_origin',
    'Numer rejestracyjny pojazdu': 'registration_number',
    'Stan': 'condition',
    'Bezwypadkowy': 'accident_free',
    'Data pierwszej rejestracji w historii pojazdu': 'first_registration_date',
    'Zarejestrowany w Polsce': 'registered_in_poland',
    'Pierwszy właściciel (od nowości)': 'first_owner',
    'Serwisowany w ASO': 'serviced_at_authorized_station',
    'Ma numer rejestracyjny': 'has_registration_number',
    'Typ złącza ładowania': 'charging_connector_type',
    'Liczba silników': 'number_of_engines',
    'Odzyskiwanie energii hamowania': 'brake_energy_recovery',
    'Liczba baterii': 'number_of_batteries',
    'equipment': 'equipment',
    'price': 'price',
    'currency': 'currency',
    'price_level': 'price_level',
    'advert_date': 'advert_date',
    'advert_id': 'advert_id',
    'description': 'description',
    'Pojemność_baterii_kWh': 'battery_capacity_kwh',
    'Autonomia_km': 'range_km',
    'engine_displacement_cm3': 'engine_displacement_cm3',
    'power_hp': 'power_hp',
    'co2_emissions_gpkm': 'co2_emissions_gpkm',
    'urban_fuel_consumption_l_per_100km': 'urban_fuel_consumption_l_per_100km',
    'extraurban_fuel_consumption_l_per_100km': 'extraurban_fuel_consumption_l_per_100km',
    'mileage_km': 'mileage_km',
    'average_energy_consumption_kwh_per_100km': 'average_energy_consumption_kwh_per_100km',
    'battery_health_percent': 'battery_health_percent',
    'max_electric_power_hp': 'max_electric_power_hp'
}

df.rename(columns=column_mapping, inplace=True)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 226088 entries, 0 to 270053
Data columns (total 44 columns):
 #   Column                                    Non-Null Count   Dtype         
---  ------                                    --------------   -----         
 0   make                                      226086 non-null  object        
 1   model                                     226088 non-null  object        
 2   version                                   154783 non-null  object        
 3   color                                     226088 non-null  object        
 4   number_of_doors                           225263 non-null  object        
 5   number_of_seats                           211394 non-null  object        
 6   production_year                           226088 non-null  int64         
 7   generation                                163504 non-null  object        
 8   fuel_type                                 226088 non-null  object        
 9   body_type           

In [ ]:
df[df['battery_capacity_kwh'].notnull()]['battery_capacity_kwh'].unique()

In [25]:
df['serviced_at_authorized_station'] = df['serviced_at_authorized_station'].replace('Bezwypadkowy', np.nan)

In [22]:
for col in ['accident_free','registered_in_poland','first_owner',
            'serviced_at_authorized_station','has_registration_number',
            'brake_energy_recovery']:
    print(col, df[col].unique())

accident_free ['Tak' <NA>]
registered_in_poland ['Tak' <NA>]
first_owner ['Tak' <NA>]
serviced_at_authorized_station ['Tak' <NA> 'Bezwypadkowy']
has_registration_number ['Tak' <NA>]
brake_energy_recovery [<NA> 'Nie' 'Tak']


In [26]:
dtype_conversion = {
    'number_of_doors': 'Int8',
    'number_of_seats': 'Int8',
    'production_year': 'Int16',
    'number_of_engines': 'Int8',
    'number_of_batteries': 'Int8',
    'mileage_km': 'Int32',
    'advert_id': 'Int64',

    'price': 'float32',
    'battery_capacity_kwh': 'float32',
    'range_km': 'float32',
    'power_hp': 'float32',
    'co2_emissions_gpkm': 'float32',
    'urban_fuel_consumption_l_per_100km': 'float32',
    'extraurban_fuel_consumption_l_per_100km': 'float32',
    'average_energy_consumption_kwh_per_100km': 'float32',
    'battery_health_percent': 'float32',
    'max_electric_power_hp': 'float32',
    'engine_displacement_cm3': 'float32',

    'accident_free': 'boolean',
    'registered_in_poland': 'boolean',
    'first_owner': 'boolean',
    'serviced_at_authorized_station': 'boolean',
    'has_registration_number': 'boolean',
    'brake_energy_recovery': 'boolean',

    'make': 'category',
    'model': 'category',
    'version': 'category',
    'color': 'category',
    'generation': 'category',
    'fuel_type': 'category',
    'body_type': 'category',
    'color_type': 'category',
    'transmission': 'category',
    'drive_type': 'category',
    'country_of_origin': 'category',
    'condition': 'category',
    'charging_connector_type': 'category',
    'currency': 'category',
    'price_level': 'category',
}

date_columns = ['first_registration_date', 'advert_date']

nullable_float_types = ['float32']
nullable_int_types = ['Int8', 'Int16', 'Int32', 'Int64']

missing_columns = []

for col, target_type in dtype_conversion.items():
    if col not in df.columns:
        missing_columns.append(col)
        continue

    if target_type in nullable_float_types:
        df[col] = pd.to_numeric(df[col], errors='coerce').astype(target_type)
    elif target_type in nullable_int_types:
        df[col] = pd.to_numeric(df[col], errors='coerce').astype(target_type)
    elif target_type == 'boolean':
        # map string 'Tak' / 'Nie' -> True / False if needed
        df[col] = df[col].replace('Tak', True).replace('Nie', False)
        df[col] = df[col].astype('boolean')
    else:
        df[col] = df[col].astype(target_type)

# Przetwarzanie kolumn datowych
for col in date_columns:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    else:
        missing_columns.append(col)

# Raport
if missing_columns:
    print("⚠️ Brakujące kolumny w DataFrame (nie zostały przekonwertowane):")
    for col in missing_columns:
        print(f" - {col}")
else:
    print("✅ Wszystkie kolumny zostały przekonwertowane pomyślnie.")

✅ Wszystkie kolumny zostały przekonwertowane pomyślnie.


In [27]:
df[df['equipment'].notnull()]['equipment'].head(10)

0     Audio i multimedia|Interfejs Bluetooth|Radio|Z...
1     Audio i multimedia|Android Auto|Interfejs Blue...
2     Audio i multimedia|Apple CarPlay|Android Auto|...
3     Audio i multimedia|Apple CarPlay|Android Auto|...
4     Audio i multimedia|Radio|Gniazdo USB|System na...
5     Audio i multimedia|Interfejs Bluetooth|Radio|G...
7     Audio i multimedia|Apple CarPlay|Android Auto|...
8     Audio i multimedia|Apple CarPlay|Android Auto|...
9     Audio i multimedia|Apple CarPlay|Android Auto|...
10    Audio i multimedia|Apple CarPlay|Android Auto|...
Name: equipment, dtype: object

In [29]:
df[df['equipment'].notnull()]['equipment'].unique()

array(['Audio i multimedia|Interfejs Bluetooth|Radio|Zestaw głośnomówiący|Komfort i dodatki|Klimatyzacja automatyczna|Podgrzewany fotel kierowcy|Podgrzewany fotel pasażera|Siedzenie z pamięcią ustawienia|Systemy wspomagania kierowcy|Tempomat|Lusterka boczne ustawiane elektrycznie|Podgrzewane lusterka boczne|Wspomaganie ruszania pod górę- Hill Holder|Spryskiwacze reflektorów|System Start/Stop|Elektroniczna kontrola ciśnienia w oponach|Wspomaganie kierownicy|Osiągi i tuning|Felgi stalowe|Bezpieczeństwo|ABS|ESP|Elektroniczny system rozdziału siły hamowania|System powiadamiania o wypadku|Poduszka powietrzna kierowcy|Poduszka powietrzna pasażera|Poduszka kolan pasażera|Kurtyny powietrzne - przód|Boczne poduszki powietrzne - przód|Kurtyny powietrzne - tył|Isofix (punkty mocowania fotelika dziecięcego)',
       'Audio i multimedia|Android Auto|Interfejs Bluetooth|Radio|Zestaw głośnomówiący|Gniazdo USB|Ładowanie bezprzewodowe urządzeń|System nawigacji satelitarnej|System nagłośnienia|Ekran dot

In [ ]:
# Zakładamy, że df już istnieje i zawiera kolumnę 'equipment'

# 1. Dodaj tymczasowy ID (local_id)
df = df.reset_index(drop=True).copy()
df['local_id'] = df.index + 1  # numerujemy od 1

# 2. Rozbij kolumnę 'equipment' na listy i standaryzuj (lowercase + bez duplikatów)
df['equipment_list'] = df['equipment'].fillna('').apply(
    lambda x: list(set(map(str.lower, [item.strip() for item in x.split('|') if item.strip()])))
)

# 3. Stwórz unikalne opcje wyposażenia
all_equipment = set(chain.from_iterable(df['equipment_list']))
equipment_df = pd.DataFrame(sorted(all_equipment), columns=['name'])
equipment_df.reset_index(inplace=True)
equipment_df.rename(columns={'index': 'id'}, inplace=True)
equipment_df = equipment_df.sort_values(by='name').reset_index(drop=True)

# 4. Mapuj wyposażenie do ID i buduj tabelę relacyjną listing_equipment
equipment_map = dict(zip(equipment_df['name'], equipment_df['id']))
rows = []

for _, row in df.iterrows():
    for eq in row['equipment_list']:
        eq_id = equipment_map.get(eq)
        if eq_id is not None:
            rows.append({'listing_id': row['local_id'], 'equipment_id': eq_id})

listing_equipment_df = pd.DataFrame(rows)

# 5. Zapisz pliki CSV do folderu
output_dir = r'C:\Users\Lukasz Pindus\VS Code Python\car_price_analysis\data'

listings_df = df.drop(columns=['equipment', 'equipment_list'])
listings_df.to_csv(os.path.join(output_dir, 'listings.csv'), index=False)
equipment_df.to_csv(os.path.join(output_dir, 'equipment_options.csv'), index=False)
listing_equipment_df.to_csv(os.path.join(output_dir, 'listing_equipment.csv'), index=False)

In [ ]:
#df['advert_id'] = df['advert_id'].astype('Int64')

df.groupby(['Marka pojazdu', 'Model pojazdu', 'price', 'Rok produkcji', 'Przebieg']).size().reset_index(name='counts')

In [ ]:
df_test = df.drop_duplicates()
print("Rozmiar po usunięciu pełnych duplikatów:", df_test.shape)

In [ ]:
df_clean = df.drop_duplicates(subset=['Marka pojazdu', 'Model pojazdu', 'price', 'Rok produkcji', 'Przebieg'])

# Teraz grupuj na df_clean, a nie df
df_clean.groupby(['Marka pojazdu', 'Model pojazdu', 'price', 'Rok produkcji', 'Przebieg']).size().reset_index(name='counts')
